# Tutorial 4 - Pioneer
This tutorial models a potential design for a WEC used to provide power to instruments in the [Pioneer Central Surface Mooring System](https://oceanobservatories.org/site/cp01cnsm/) within the [Coastal Pioneer Array](https://oceanobservatories.org/array/coastal-pioneer-array/) off the coast of New England. This system has instrumentation to measure various meteorological, surface, near-surface, and seabed phenomena on the Continental Shelf-Slope and transmit back to shore.

Unlike previous tutorials, this tutorial does not include an outer optimization loop. Instead, the focus here is on effectively modeling an optimization problem for the control states of a unique WEC archetype with an pitch resonator PTO with several co-dependent components. Topics include:

* Expanding the control state vector
* Custom PTO physics
* Modeling non-hydrodynamic degrees of freedom
<p><img src=https://oceanobservatories.org/wp-content/uploads/2015/09/Central-Surface-Mooring-Array-01-1.png alt="Diagram of the Pioneer device" width="307">

In [ ]:
import os
import capytaine as cpy
import autograd.numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import block_diag

import wecopttool as wot

## set colorblind-friendly colormap for plots
plt.style.use('tableau-colorblind10')

## Model definition

### Geometry
For purposes of getting the required hydrodynamic coefficients to model the Pioneer design, we only need to be concerned with creating a mesh of the buoy geometry. The buoy has the same general shape as the WaveBot used in Tutorial 1, so we will use the build-in `geom.WaveBot` object and change the dimensions to simplify the mesh creation process.

*~add two engineering drawings*

In [ ]:
in2m = 0.0254 # inch -> meter conversion factor
draft = 0.5 # meters
freeboard = 40. * in2m - draft
hull_geom = wot.geom.WaveBot(r1=130./2 * in2m,
                             r2=110./2 * in2m, 
                             h1=22.679 * in2m,
                             h2=17.321 * in2m,
                             scale_factor=1,
                             freeboard=freeboard)
mesh = hull_geom.mesh(mesh_size_factor=0.5)
_ = hull_geom.plot_cross_section()

### Given properties
The Pioneer team has provided specifications for a full-sized WEC design that fits within the buoy dimensions and operating at resonance. There are three components at account for with the Pioneer model that we need to model the PTO properly:

* **Buoy**: This has the same parameters as the WaveBot used in Tutorial 1.
* **Flywheel**: The flywheel is contained within the buoy. It will rotate as the buoy rotates, though this rotation will be amplified due to the gearing connected ot the spring. Since the flywheel is contained within the buoy, it is **not** directly subject to wave forcing.
* **Spring**: The magnetic torsional spring provides positive stiffness to the flywheel. It has a linear displacement up to 45 degrees, at which point it "wraps around" and the stiffness begins decreasing. We will model the spring as linear for the time being.

In [ ]:
buoy_props = {
    'CG': 0.242, # m
    'MOI': 7484., # kg-m^2
}

flywheel_props = {
    'MOI': 25,  # kg-m^2
    'Coulomb friction': 2.0,  # N-m
    'Viscous friction': 0.02,  # N-m/rad
    'Gear ratio': 3, # -
}

spring_props = {
    'Max torque': 750, # N-m
    'Max displacement': np.deg2rad(45.0), # rad
}

### Pitch resonator

This system uses a unique PTO system dependent on the relative pitching rotation of the buoy and the flywheel (subject to the connected spring and gearing). The system can be modeled as a coupled spring-mass-damper system, as shown below (note that all of these values are rotational and are modeled here linearly for simplicity): 

*~add dynamic diagram*

This corresponds to the following dynamics equations:

(1) $$ [J_{buoy} + J(\omega))] \ddot{\theta}_{buoy} = T_{hydro} - K_{hydro}\theta_{buoy} - B(\omega)\dot{\theta}_{buoy} - K_{spring}N^2(\theta_{buoy} - \theta_{flywheel}) - (B_{PTO} + B_{fric})(\dot{\theta}_{buoy} - \dot{\theta}_{flywheel}) $$

(2) $$ J_{flywheel} \ddot{\theta}_{flywheel} = K_{spring}N^2(\theta_{buoy} - \theta_{flywheel}) + (B_{PTO} + B_{fric})(\dot{\theta}_{buoy} - \dot{\theta}_{flywheel}) $$

We will explore how to capture these dynamics in WecOptTool in the PTO section below.

## Hydrodynamics and hydrostatics
### Floating body
As mentioned above, the `FloatingBody` object in Capytaine only needs to model the buoy, since no other components are being excited by the waves.

As a quick check, the hydrostatic stiffness of the device was experimentally determined by the Pioneer team by ~~~. The hydrostatic stiffness calculated by Capytaine is reasonably close to this value.

In [ ]:
pnr_fb = cpy.FloatingBody.from_meshio(mesh, name="Pioneer")
pnr_fb.add_rotation_dof(name='Pitch')
pnr_fb.center_of_mass = np.array([0., 0., buoy_props['CG']])
pnr_fb.rotation_center = pnr_fb.center_of_mass
ndof = pnr_fb.nb_dofs
pnr_fb.show_matplotlib()

In [ ]:
rho = 1025. # kg/m^3
f1 = 0.025 # Hz
nfreq = 25
freq = wot.frequency(f1, nfreq, False) # False -> no zero frequency


file_name = "data/pioneer_bem_25freq.nc"
if os.path.exists(file_name):
    bem_data = wot.read_netcdf(file_name)
else:    
    bem_data = wot.run_bem(pnr_fb, freq)
    wot.write_netcdf(file_name, bem_data)

pnr_fb.keep_immersed_part()
k_buoy = pnr_fb.compute_hydrostatic_stiffness(rho=rho).values.squeeze()
k_spring = spring_props['Max torque'] / spring_props['Max displacement']
print(f'Hydrostatic stiffness from Capytaine: {k_buoy} N-m/rad')
print('Hydrostatic stiffness from experiment: 37204 N-m/rad')

omega = bem_data.omega.values
omega_r = np.sqrt(k_buoy / buoy_props['MOI']) # resonance frequency

As with previous tutorials, we will plot the hydrodynamic coefficients to confirm that we are properly capturing the full excitation range of the buoy.

In [ ]:
# plots
fig_am, ax_am = plt.subplots(tight_layout=True, sharex=True)
fig_rd, ax_rd = plt.subplots(tight_layout=True, sharex=True)
fig_ex, ax_ex = plt.subplots(tight_layout=True, sharex=True)

# Excitation
np.abs(bem_data.diffraction_force.sel(influenced_dof='Pitch')).plot(
    ax=ax_ex, linestyle='dashed', label='Diffraction force')
np.abs(bem_data.Froude_Krylov_force.sel(influenced_dof='Pitch')).plot(
    ax=ax_ex, linestyle='dashdot', label='Froude-Krylov force')
ex_handles, ex_labels = ax_ex.get_legend_handles_labels()
ax_ex.set_xlabel(f'$\omega$', fontsize=10)
ax_ex.set_title('Wave Excitation Coefficients', fontweight='bold')
fig_ex.legend(ex_handles, ex_labels, loc='center right', frameon=False)

# Added mass
bem_data.added_mass.sel(
    radiating_dof='Pitch', influenced_dof='Pitch').plot(ax=ax_am)
ax_am.set_xlabel(f'$\omega$', fontsize=10)
ax_am.set_title('Added Mass Coefficients', fontweight='bold')

# Radiation damping
bem_data.radiation_damping.sel(
    radiating_dof='Pitch', influenced_dof='Pitch').plot(ax=ax_rd)
ax_rd.set_xlabel(f'$\omega$', fontsize=10)
ax_rd.set_title('Radiation Damping Coefficients', fontweight='bold')

## PTO
Analyzing the system as a coupled mass-spring-damper as above, we can calculate the generator velocity to be the difference between the buoy velocity and the geared flywheel velocity. Because the velocity needed for the power equations are dependent on two nonlinearly coupled components (one of which is not directly affected by hydrodynamics) we cannot use the `PTO` module we have used in previous tutorials. Instead, we will make two adjustments to our model compared to previous tutorials:

1. We will define the flywheel motions as extra variables in the `x_opt` vector
2. We will manually define the PTO equations.

### Capturing flywheel motions
In previous tutorials, all the relevant WEC dynamics have been captured in the `x_wec` vector. However, `x_wec` assumes all the degrees of freedom are excited by waves, which is not the case for the flywheel in the Pioneer model.

The `x_opt` vector has previously only been used to model the PTO force from an unstructured controller at each Fourier coefficient. However, there is no limit to what `x_opt` can include; it can really include anything that is needed to model the PTO control. Since we need to capture the flywheel dynamics to properly model the PTO, we will define `x_opt` to be twice as long as previous tutorials. The first half will continue to be used for the PTO force from the unstructured controller, of length `2 * nfreq`. The second half will the position states of the flywheel at each Fourier coefficient, also of length `2 * nfreq`.

Note in the code further down that `x_opt` will be spliced as either `x_opt[:nstate_pto]` to exclude the flywheel position states, or `x_opt[nstate_pto:]` to exclude the PTO force states. Remember, `x_opt` can be defined arbitrarily, and specific meanings to the states are given elsewhere in the model.

In [ ]:
nstate_pto = 2 * nfreq # PTO forces
nstate_fw = 2 * nfreq  # Flywheel positions
nstate_opt = nstate_pto + nstate_fw

### Manually defining PTO equations

In [ ]:
def block(re, im): return np.array([[re, -im], [im, re]])
for idof in range(2):
    for jdof in range(2):
        Zp = abcd[idof, jdof, :]
        re = np.real(Zp)
        im = np.imag(Zp)
        # Exclude the sine component of the 2-point wave
        blocks = [block(ire, iim) for (ire, iim) in zip(re[:-1], im[:-1])]
        # re[0] added for the zero frequency power loss (DC), could be re[n]
        blocks = [re[0]] + blocks + [re[-1]]
        if jdof==0:
            row = block_diag(*blocks)
        else:
            row = np.hstack([row, block_diag(*blocks)])
    if idof==0:
        transfer_mat = row
    else:
        transfer_mat = np.vstack([transfer_mat, row])

In [ ]:
def x_rel(wec, x_wec, x_opt):
    pos_wec = wec.vec_to_dofmat(x_wec)
    pos_fw = wec.vec_to_dofmat(x_opt[nstate_pto:])
    return pos_wec - pos_fw

def rel_position(wec, x_wec, x_opt, waves, nsubsteps=1):
    pos_rel = x_rel(wec, x_wec, x_opt)
    time_matrix = wec.time_mat_nsubsteps(nsubsteps)
    return np.dot(time_matrix, pos_rel)

def rel_velocity(wec, x_wec, x_opt, waves, nsubsteps=1):
    pos_rel = x_rel(wec, x_wec, x_opt)
    vel_rel = np.dot(wec.derivative_mat, pos_rel)
    time_matrix = wec.time_mat_nsubsteps(nsubsteps)
    return np.dot(time_matrix, vel_rel)

def f_motor(wec, x_wec, x_opt, waves, nsubsteps=1):
    motor = np.reshape(x_opt[:nstate_pto], (-1, ndof), order='F')
    time_matrix = wec.time_mat_nsubsteps(nsubsteps)
    return np.dot(time_matrix, motor)

def mechanical_power(wec, x_wec, x_opt, waves, nsubsteps=1):
    force_td = f_motor(wec, x_wec, x_opt, waves, nsubsteps)
    vel_td = rel_velocity(wec, x_wec, x_opt, waves, nsubsteps)
    return vel_td * force_td

def power(wec, x_wec, x_opt, waves, nsubsteps=1):
    if transfer_mat is not None:
        q1_td = rel_velocity(wec, x_wec, x_opt, waves)
        e1_td = f_motor(wec, x_wec, x_opt, waves)
        q1 = wot.complex_to_real(wec.td_to_fd(q1_td, False))
        e1 = wot.complex_to_real(wec.td_to_fd(e1_td, False))
        vars_1 = np.hstack([q1, e1])
        vars_1_flat = wec.dofmat_to_vec(vars_1)
        vars_2_flat = np.dot(transfer_mat, vars_1_flat)
        vars_2 = wot.vec_to_dofmat(vars_2_flat, 2)
        q2 = vars_2[:, 0]
        e2 = vars_2[:, 1]
        time_mat = wec.time_mat_nsubsteps(nsubsteps)
        q2_td = np.dot(time_mat, q2)
        e2_td = np.dot(time_mat, e2)
    else:
        q2_td = rel_velocity(wec, x_wec, x_opt, waves, nsubsteps)
        e2_td = f_motor(wec, x_wec, x_opt, waves, nsubsteps)
    return q2_td * e2_td

def energy(wec, x_wec, x_opt, waves, nsubsteps=1):
    power_td = power(wec, x_wec, x_opt, waves, nsubsteps)
    return np.sum(power_td) * wec.dt/nsubsteps

def average_power(wec, x_wec, x_opt, waves, nsubsteps=1):
    e = energy(wec, x_wec, x_opt, waves, nsubsteps)
    return e / wec.tf

### PTO Impedance

Using same impedance matrix as WaveBot, except replace gear ratio with the one determined from the spring stiffness and buoy moment of inertia + natural frequency.

In [ ]:
n = flywheel_props['Gear ratio']
torque_constant = 3.512
winding_resistance = 0.304
winding_inductance = 0.0
drivetrain_inertia = 0.0
drivetrain_friction = 0.0
drivetrain_stiffness = 0.0

drivetrain_impedance = (1j*omega*drivetrain_inertia + 
                        drivetrain_friction + 
                        1/(1j*omega)*drivetrain_stiffness) 

winding_impedance = winding_resistance + 1j*omega*winding_inductance


z_11 = np.array([[-1* n**2 * drivetrain_impedance]])
off_diag = np.sqrt(3.0/2.0) * torque_constant * n
z_12 = np.array([[-1*(off_diag+0j) * np.ones(omega.shape)]])
z_21 = np.array([[-1*(off_diag+0j) * np.ones(omega.shape)]])
z_22 = np.array([[winding_impedance]])
z_12_inv = np.linalg.inv(z_12.T).T

mmult = lambda a,b: np.einsum('mnr,mnr->mnr', a, b)
abcd_11 = -1 * mmult(z_12_inv, z_11)
abcd_12 = z_12_inv
abcd_21 = z_21 - mmult(z_22, mmult(z_12_inv, z_11))
abcd_22 = mmult(z_22, z_12_inv)

row_1 = np.hstack([abcd_11, abcd_12])
row_2 = np.hstack([abcd_21, abcd_22])
abcd = np.vstack([row_1, row_2])

### Constraints
 * **Peak torque** - Peak torque
 * **Peak RMS speed** - Peak RMS speed

In [ ]:
torque_peak_max = 50.
permitted_rms_speed = 3300 * 2*np.pi / 60 # rad/s
def const_peak_torque_pto(wec, x_wec, x_opt, waves, nsubsteps=2):
    torque = f_motor(wec, x_wec, x_opt, waves, nsubsteps)
    return torque_peak_max - np.abs(torque.flatten())

def const_rms_velocity_pto(wec, x_wec, x_opt, waves, nsubsteps=2):
    pos = wec.vec_to_dofmat(x_opt[nstate_pto:])
    pto_vel = np.dot(wec.derivative_mat, pos)
    rms_vel = np.sqrt(np.mean(pto_vel**2))
    return permitted_rms_speed - rms_vel

### Additional forces
 * **Flywheel friction** - Flywheel friction
 * **Magnetic spring** - Magnetic spring
 * **Flywheel inertia** - Not really a force, but captured here for the flywheel residual function below

In [ ]:
def f_spring(wec, x_wec, x_opt, waves, nsubsteps=1):
    rel_pos = rel_position(wec, x_wec, x_opt, waves, nsubsteps)
    return -k_spring * (1/n)**2 * rel_pos

def f_friction(wec, x_wec, x_opt, waves, nsubsteps=1):
    rel_vel = rel_velocity(wec, x_wec, x_opt, waves, nsubsteps)
    return (
        np.tanh(rel_vel) * flywheel_props['Coulomb friction']
      + rel_vel * flywheel_props['Viscous friction']
    )

def fw_inertia(wec, x_wec, x_opt, waves, nsubsteps=1):
    pos_fw = wec.vec_to_dofmat(x_opt[nstate_pto:])
    acc_fw = np.dot(wec.derivative2_mat, pos_fw)
    time_matrix = wec.time_mat_nsubsteps(nsubsteps)
    acc_fw = np.dot(time_matrix, acc_fw)
    return flywheel_props['MOI'] * acc_fw

#### Residual
Equality constraint of Newton's 2nd law on the flywheel (i.e. $r(x) = I \alpha - \tau = 0 $). Same basic structure as residual for the WEC dynamics as described in the [WecOptTool theory documentation](https://sandialabs.github.io/WecOptTool/theory.html).

In [ ]:
def const_flywheel_resid(wec, x_wec, x_opt, waves):
    return (fw_inertia(wec, x_wec, x_opt, waves)
          + f_spring(wec, x_wec, x_opt, waves)
          - f_friction(wec, x_wec, x_opt, waves)
          + f_motor(wec, x_wec, x_opt, waves)
          ).flatten()
    

### WEC object
Finally, we can use all the different components we've developed thus far to construct a `WEC` object:

In [ ]:
f_add = {
    'Spring': f_spring,
    'Motor': f_motor,
}

constraints = [
    {'type': 'ineq', 'fun': const_peak_torque_pto},
    {'type': 'ineq', 'fun': const_rms_velocity_pto},
    {'type': 'eq', 'fun': const_flywheel_resid},
]

In [ ]:
wec = wot.WEC.from_bem(bem_data,
                       inertia_matrix=np.array([[buoy_props['MOI']]]),
                       hydrostatic_stiffness=k_buoy,
                       f_add=f_add,
                       constraints=constraints,
                       dof_names=bem_data.influenced_dof.values,
                        )

## Waves
A regular wave will allow us to get a good initial understanding of the optimal control trajectory.
Note that we'll want to choose a wave frequency that is within the frequency array we used to calculate the hydrodynamic data.

In [ ]:
amplitude = 0.15
# wavefreq =  1/5.3
wavefreq =  omega_r / (2*np.pi)
phase = 0
wavedir = 0
waves = wot.waves.regular_wave(f1, nfreq, wavefreq, amplitude, phase, wavedir)
# spectrum = lambda f: wot.waves.jonswap_spectrum(freq=f, fp=wavefreq, hs=2*amplitude)
# efth = wot.waves.omnidirectional_spectrum(f1, nfreq, spectrum, "JONSWAP")
# waves = wot.waves.long_crested_wave(efth)
# efth.plot(marker='.')

## Solve
Note the size of the `nstate_opt` variable compared to previous tutorials. The first half of the variable includes the PTO force states at each Fourier component, as in previous tutorials (the `nstate_pto` component). The second half is the flywheel positions at each Fourier component, which is basically an analog to `wec.nstate_wec`.

In [ ]:
obj_fun = average_power
nstate_opt = nstate_pto + wec.nstate_wec
options = {'maxiter': 500}
results = wec.solve(
    waves,
    obj_fun,
    nstate_opt,
    x_wec_0=np.ones(wec.nstate_wec)*0.8,
    x_opt_0=np.ones(nstate_opt),
    scale_x_wec=1e1,
    scale_x_opt=1e-2,
    scale_obj=1e-2,
    optim_options=options
)

## Post-process and plotting

In [ ]:
results.fun

In [ ]:
nsubsteps = 5
wec_fdom, wec_tdom = wec.post_process(results, waves, nsubsteps=nsubsteps)
force_excitation = (wec_tdom.force.sel(type='Froude_Krylov') + 
                    wec_tdom.force.sel(type='diffraction'))
exc_torque = force_excitation.squeeze().values

In [ ]:
x_wec, x_opt = wot.decompose_state(results.x, 1, nfreq)
pto_pos = rel_position(wec, x_wec, x_opt, waves, nsubsteps) / n**2
pto_vel = rel_velocity(wec, x_wec, x_opt, waves, nsubsteps)
pto_force = f_motor(wec, x_wec, x_opt, waves, nsubsteps)
pto_force_fd = wec.td_to_fd(pto_force[::nsubsteps])
pto_mech_power = mechanical_power(wec, x_wec, x_opt, waves, nsubsteps)
fw_pos = np.dot(wec.time_mat_nsubsteps(nsubsteps), x_opt[nstate_pto:])

In [ ]:
fig, ax = plt.subplots(nrows=6, sharex=True, figsize=(12, 12))
t = wec_tdom.time.values

# Excitation
ax[0].plot(t, comp_pto_trq, label='Reference')
ax[0].plot(t, pto_force, linestyle='dashed', label='WecOptTool')
ax[0].set_ylabel('Torque [N-rad]')
ax[0].set_title('PTO Torque')
ax[0].legend()

# Buoy Position
ax[1].plot(t, comp_buoy_pos, label='Reference')
(wec_tdom.pos*180/np.pi).plot(ax=ax[1], hue='influenced_dof', linestyle='dashed', label='WecOptTool')
ax[1].set_title('Buoy Position')
ax[1].set_ylabel('Position [deg]')
ax[1].legend()

# Flywheel Position
ax[2].plot(t, comp_fw_pos, label='Reference')
ax[2].plot(t, fw_pos*180/np.pi, linestyle='dashed', label='WecOptTool')
ax[2].set_title('Flywheel Position')
ax[2].set_ylabel('Position [deg]')
ax[2].legend()

# PTO Position
ax[3].plot(t, comp_pto_pos, label='Reference')
ax[3].plot(t, pto_pos*180/np.pi, linestyle='dashed', label='WecOptTool')
# ax[3].plot(t, np.ones(len(t))*45)
ax[3].set_title('PTO Position')
ax[3].set_ylabel('Position [deg]')
ax[3].legend()

# PTO Velocity
ax[4].plot(t, comp_gen_vel, label='Reference')
ax[4].plot(t, pto_vel, linestyle='dashed', label='WecOptTool')
ax[4].set_title('PTO Velocity')
ax[4].set_ylabel('Velocity [rad/s]')
ax[4].legend()

# Mechanical Power
ax[5].plot(t, comp_mech_power, label='Reference')
ax[5].plot(t, pto_mech_power, linestyle='dashed', label='WecOptTool')
ax[5].set_title('Power Output')
ax[5].set_ylabel('Power [W]')
ax[5].legend()

for axi in ax:
    axi.grid()
    axi.label_outer()
    axi.autoscale(axis='x', tight=True)

In [ ]:
plt.plot(wec.omega, pto_force_fd)